<a href="https://colab.research.google.com/github/maasanmrs/AIpractice/blob/maasanmrs-patch-1/BitcoinBot%E6%9B%B4%E6%96%B0%E7%89%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PDFを投げるとその文章を読み込み、その文章に関しての質問に答えてくれるチャットボット

PDFからテキストデータを読み込み

In [ ]:
!pip install -U langchain-community # Install the langchain community package

#必要なライブラリインポート
# Embedding用
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
# テキストファイルを読み込む
from langchain.document_loaders import TextLoader

  Using cached langchain-0.2.16-py3-none-any.whl.metadata (7.1 kB)
Using cached langchain-0.2.16-py3-none-any.whl (1.0 MB)
  Attempting uninstall: langchain
    Found existing installation: langchain 0.0.125
    Uninstalling langchain-0.0.125:
      Successfully uninstalled langchain-0.0.125


In [ ]:
!pip install langchain==0.0.125 openai==0.27.2 chromadb==0.3.14 pypdf==3.7.0 tiktoken==0.3.3 gradio==3.23

  Using cached langchain-0.0.125-py3-none-any.whl.metadata (8.0 kB)
Using cached langchain-0.0.125-py3-none-any.whl (443 kB)
  Attempting uninstall: langchain
    Found existing installation: langchain 0.2.16
    Uninstalling langchain-0.2.16:
      Successfully uninstalled langchain-0.2.16
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.2.16 requires langchain<0.3.0,>=0.2.16, but you have langchain 0.0.125 which is incompatible.


In [ ]:
#必要なライブラリのインストール
!pip install langchain
!pip install -q -U google-generativeai

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import VectorDBQA, RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader, PyPDFLoader

import os
#Geminiに変更
#os.environ["OPENAI_API_KEY"] = "***"

In [ ]:
#Google Driveへのアクセス許可
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 環境変数の準備
import google.generativeai as genai
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
loader = PyPDFLoader('/content/drive/MyDrive/19_28The Bitcoin Standard.pdf')
documents = loader.load()

In [ ]:
import pandas as pd # Import the pandas library and give it the alias 'pd'


In [ ]:
df = pd.DataFrame(documents)

In [ ]:
df.head()

,0,1,2,3
0,"(id, None)","(metadata, {'source': '/content/drive/MyDrive/...","(page_content, Trim Size: 6in x 9in Ammous c01...","(type, Document)"
1,"(id, None)","(metadata, {'source': '/content/drive/MyDrive/...","(page_content, Trim Size: 6in x 9in Ammous c01...","(type, Document)"
2,"(id, None)","(metadata, {'source': '/content/drive/MyDrive/...","(page_content, Trim Size: 6in x 9in Ammous c01...","(type, Document)"
3,"(id, None)","(metadata, {'source': '/content/drive/MyDrive/...","(page_content, Trim Size: 6in x 9in Ammous c01...","(type, Document)"
4,"(id, None)","(metadata, {'source': '/content/drive/MyDrive/...","(page_content, Trim Size: 6in x 9in Ammous c01...","(type, Document)"


In [ ]:
df.columns

RangeIndex(start=0, stop=4, step=1)

In [ ]:
df=df.drop(0,axis=1).drop(3,axis=1)
DF1=df

In [ ]:
df.head()

,1,2
0,"(metadata, {'source': '/content/drive/MyDrive/...","(page_content, Trim Size: 6in x 9in Ammous c01..."
1,"(metadata, {'source': '/content/drive/MyDrive/...","(page_content, Trim Size: 6in x 9in Ammous c01..."
2,"(metadata, {'source': '/content/drive/MyDrive/...","(page_content, Trim Size: 6in x 9in Ammous c01..."
3,"(metadata, {'source': '/content/drive/MyDrive/...","(page_content, Trim Size: 6in x 9in Ammous c01..."
4,"(metadata, {'source': '/content/drive/MyDrive/...","(page_content, Trim Size: 6in x 9in Ammous c01..."


## データの加工
apply関数を使用しdfをsourceとcontentsのみに変換
また、contentsの序盤60文字部分ほどの不要な部分を削除


In [ ]:
DF1[1]=df[1].apply(lambda x: x[1]['source'])
DF1[2]=df[2].apply(lambda x: x[1][65:])

In [ ]:
DF1=DF1.rename(columns={1: 'source',2:'contents'})
DF1


,source,contents
0,/content/drive/MyDrive/19_28The Bitcoin Standa...,k\nk k\nkChapter 1\nMoney\nBitcoin is the new...
1,/content/drive/MyDrive/19_28The Bitcoin Standa...,k\nk k\nk2 THE BITCOIN STANDARD\nuseinexcepti...
2,/content/drive/MyDrive/19_28The Bitcoin Standa...,"k\nk k\nkMoney 3\nnaturally emerge, if only b..."
3,/content/drive/MyDrive/19_28The Bitcoin Standa...,k\nk k\nk4 THE BITCOIN STANDARD\nsold on the ...
4,/content/drive/MyDrive/19_28The Bitcoin Standa...,k\nk k\nkMoney 5\nFor the good to maintain it...
5,/content/drive/MyDrive/19_28The Bitcoin Standa...,k\nk k\nk6 THE BITCOIN STANDARD\nincreasedwil...
6,/content/drive/MyDrive/19_28The Bitcoin Standa...,k\nk k\nkMoney 7\nsalable forms of money. But...
7,/content/drive/MyDrive/19_28The Bitcoin Standa...,k\nk k\nk8 THE BITCOIN STANDARD\ndominatethem...
8,/content/drive/MyDrive/19_28The Bitcoin Standa...,k\nk k\nkMoney 9\ncanbecome.Producerscanspeci...
9,/content/drive/MyDrive/19_28The Bitcoin Standa...,0 k\nk k\nk10 THE BITCOIN STANDARD\nand cattle...


データセット完成  

##embedding
以下indexごとにembeddingをする

In [ ]:
!pip install -q langchain

import pathlib
import textwrap

import google.generativeai as genai
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Assuming 'texts' is your list of strings, convert them to Document objects
#documents = [Document(page_content=t) for t in texts]

#text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
#split_texts = text_splitter.split_documents(documents) # Pass the list of Document objects

In [ ]:
# Used to securely store your API key
from google.colab import userdata

In [ ]:
embeddings = genai.embed_content(
    model="models/embedding-001",
    content=[Document(page_content=text).page_content for text in DF1['contents']],  # Create Document objects and extract text for each element in DF1[0]
    task_type="retrieval_document",
    title="The Bitcoin Standard"
)

In [ ]:
#データフレームに見やすく格納
DF1['Embeddings']=embeddings['embedding']
df=DF1
df

,source,contents,Embeddings
0,/content/drive/MyDrive/19_28The Bitcoin Standa...,k\nk k\nkChapter 1\nMoney\nBitcoin is the new...,"[0.03475657, -0.033548303, -0.05403553, -0.003..."
1,/content/drive/MyDrive/19_28The Bitcoin Standa...,k\nk k\nk2 THE BITCOIN STANDARD\nuseinexcepti...,"[0.026462108, -0.04585655, -0.0677653, -0.0095..."
2,/content/drive/MyDrive/19_28The Bitcoin Standa...,"k\nk k\nkMoney 3\nnaturally emerge, if only b...","[0.009874306, -0.061145376, -0.03727729, -0.00..."
3,/content/drive/MyDrive/19_28The Bitcoin Standa...,k\nk k\nk4 THE BITCOIN STANDARD\nsold on the ...,"[0.0032092752, -0.04940647, -0.04716462, 0.001..."
4,/content/drive/MyDrive/19_28The Bitcoin Standa...,k\nk k\nkMoney 5\nFor the good to maintain it...,"[0.004497256, -0.030958984, -0.025018344, -0.0..."
5,/content/drive/MyDrive/19_28The Bitcoin Standa...,k\nk k\nk6 THE BITCOIN STANDARD\nincreasedwil...,"[0.024855549, -0.05039598, -0.028572671, -0.00..."
6,/content/drive/MyDrive/19_28The Bitcoin Standa...,k\nk k\nkMoney 7\nsalable forms of money. But...,"[0.017566357, -0.06787652, -0.013745709, 0.009..."
7,/content/drive/MyDrive/19_28The Bitcoin Standa...,k\nk k\nk8 THE BITCOIN STANDARD\ndominatethem...,"[0.025738124, -0.062507994, -0.03079968, -0.00..."
8,/content/drive/MyDrive/19_28The Bitcoin Standa...,k\nk k\nkMoney 9\ncanbecome.Producerscanspeci...,"[0.015016648, -0.026907299, -0.044630088, 0.01..."
9,/content/drive/MyDrive/19_28The Bitcoin Standa...,0 k\nk k\nk10 THE BITCOIN STANDARD\nand cattle...,"[0.019130252, -0.0299127, -0.018654594, 0.0008..."


embedding終了

Chatbot用のUIの実装とChatbotに入力された質問をembedingして、類似した文章を探し、一番類似した文章を入力としてGeminiに回答を生成させる。フロントはサイトの流用


#参考にしたもの
https://ai.google.dev/gemini-api/tutorials/document_search?hl=ja

https://youtu.be/eCtHVmXcXMs?si=4uF4qrCmaZ2XKz_H

In [ ]:
import numpy as np

In [ ]:
from langchain import PromptTemplate

In [ ]:
import gradio as gr

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))


def add_text(history, text):
    history = history + [(text, None)]
    return history, ""

def bot(history):

    query = history[-1][0]
    #query = prompt.format(question=query)
    model = 'models/embedding-001'

    request = genai.embed_content(model=model,
                                  content=query,
                                  task_type="retrieval_query")

    def find_best_passage(query, dataframe):
        """
        Compute the distances between the query and each document in the dataframe
        using the dot product.
        """
        query_embedding = genai.embed_content(model=model,
                                        content=query,
                                        task_type="retrieval_query")
        dot_products = np.dot(np.stack(df['Embeddings']), query_embedding["embedding"])
        idx = np.argmax(dot_products)
        return dataframe.iloc[idx]['contents'] # Return text from index with max value

    passage = find_best_passage(query, df)
    passage


    def make_prompt(query, relevant_passage):
      escaped = relevant_passage#.replace("'", "").replace('"', "").replace("\n", " ")
      prompt = textwrap.dedent("""You are a helpful and informative bot that answers questions using text from the reference passage included below. \
                                 Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. \
                                  However, you are talking to a non-technical audience, so be sure to break down complicated concepts and \
                                    strike a friendly and converstional tone. \
                                 If the passage is irrelevant to the answer, you may ignore it.
                                  QUESTION: '{query}'
                                    PASSAGE: '{relevant_passage}'

                                  ANSWER:
                                """).format(query=query, relevant_passage=escaped)

      return prompt

    prompt = make_prompt(query, passage)
    print(prompt)

    model = genai.GenerativeModel('gemini-1.5-pro-latest')
    answer = model.generate_content(prompt).text





    #source = qa._get_docs(query)[0]
    #source_sentence = source.page_content
    #answer_source = source_sentence +"\n"+"source:"+source.metadata["source"] + ", page:" + str(source.metadata["page"])
    history[-1][1] = answer # + "\n\n情報ソースは以下です：\n" + answer_source
    return history




with gr.Blocks() as demo:
    chatbot = gr.Chatbot([], elem_id="chatbot").style(height=400)

    with gr.Row():
        with gr.Column(scale=0.6):
            txt = gr.Textbox(
                show_label=False,
                placeholder="Enter text and press enter",
            ).style(container=False)

    txt.submit(add_text, [chatbot, txt], [chatbot, txt]).then(
        bot, chatbot, chatbot
    )

demo.launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

You are a helpful and informative bot that answers questions using text from the reference passage included below.                                  Be sure to respond in a complete sentence, being comprehensive, including all relevant background information.                                   However, you are talking to a non-technical audience, so be sure to break down complicated concepts and                                     strike a friendly and converstional tone.                                  If the passage is irrelevant to the answer, you may ignore it.
                                  QUESTION: 'What is money?'
                                    PASSAGE: ' k
k k
kMoney 3
naturally emerge, if only because those who chance upon it will be far
more productive than those who do not: a single medium of exchange
(or at most a small number of media of exchange) emerges for everyone
totradetheirgoodsfor.Agoodthatassumestheroleofawidelyaccepted
medium of exchange is called money.
